### Data Ingestion Pipeline

Flightstats API connection --> 

store initial queries as samples --> 

filter by date --> 

web scrape data at hour intervals --> 

run mutliple queries from Flightstats database --> 

append output tables by time --> 

combine and store as csv

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import streamlit as st
from bs4 import BeautifulSoup
import pytz
from datetime import datetime
import re

In [ ]:
indices = np.arange(0, 1, 20)
indices = [str(i) for i in indices]
# Initialize an empty list to store dataframes
dfs_list = []
# Read the first URL separately to initialize df
df = pd.read_html('https://www.flightaware.com/live/fleet/SWA?;offset=100;order=ident;sort=ASC')[3]
dfs_list.append(df)
# Loop through the remaining URLs
for j in range(1, len(indices)):
    updated_url = f'https://www.flightaware.com/live/fleet/SWA'
    dfs = pd.read_html(updated_url)[3]
    # Ensure the columns match, you may need to manipulate dfs here
    # to match the columns and their order in the initial df
    # For example, you can use reordering or renaming columns
    dfs_list.append(dfs)

# Concatenate all the dataframes in the list
final_df = pd.concat(dfs_list, ignore_index=True)
# final_df.to_csv('Data/Extracted_flightstats_data.csv')
# sample query as HTTP requests

In [58]:
# extract appended schedule data
# data cleaning --> extract new schedule dataset
data = pd.read_csv('Data/Extracted_flightstats_data.csv')

# remove all null entry columns
columns_to_remove = ['Unnamed: 0','Airborne Southwest "Southwest" (SWA) Aircraft.6', 'Airborne Southwest "Southwest" (SWA) Aircraft.7']
data = data.drop(columns=columns_to_remove)

# drop first row
col_names = ['Flight_Number', 'Aircraft_Type','Origin_Airport', 'Destination_Airport', 'Departure_Time', 'Arrival_Time']

# create new names for columns along axis 1
data = data.drop(0)
# Assign the new column names to the DataFrame
data.columns = col_names

# extract origin and destination ITA codes
def extract_airport_code(entry):
    start = entry.find('(K') + 2  # Find the position of '(K' and move 2 characters ahead to the airport code
    end = entry.find(')', start)  # Find the closing parenthesis
    if start != -1 and end != -1:
        return entry[start:end]
    else:
        return None
    
# Apply the custom function to the 'Airport' column
data['Origin_Airport_Code'] = data['Origin_Airport'].apply(extract_airport_code)
# Remove the 'K' from the airport code

data['Origin_Airport_Code'] = data['Origin_Airport_Code'].str.replace('K', '')
# Apply the custom function to the 'Airport' column

data['Destination_Airport_Code'] = data['Destination_Airport'].apply(extract_airport_code)

# Remove the 'K' from the airport code
data['Destination_Airport_Code'] = data['Destination_Airport_Code'].str.replace('K', '')

# remove day Tue from departure and arrival time 
data['Departure_Time'] = data['Departure_Time'].str.replace(r'^\w+\s', '', regex=True)
data['Arrival_Time'] = data['Arrival_Time'].str.replace(r'^\w+\s', '', regex=True)
data['Departure_Time'] = data['Departure_Time'].str.replace(r'\xa0', ' ', regex=True)
data['Arrival_Time'] = data['Arrival_Time'].str.replace(r'\xa0', ' ', regex=True)


# Define a regular expression pattern to match the time zone code
pattern = r'([A-Z]{3})$'
# Use the str.extract method to extract the time zone codes
time_codes = data['Departure_Time'].str.extract(pattern)
time_codes_list = set(time_codes[0].tolist())
print(time_codes_list)

# split by entry in times column 
def convert_to_est(time_str, source_timezone):
    # Define a dictionary to map timezone abbreviations to pytz timezone objects
    timezone_mapping = {
        'AST': pytz.timezone('America/Puerto_Rico'),
        'CDT': pytz.timezone('America/Chicago'),
        'EDT': pytz.timezone('US/Eastern'),
        'EST': pytz.timezone('US/Eastern'),
        'HST': pytz.timezone('Pacific/Honolulu'),
        'MDT': pytz.timezone('America/Denver'),
        'MST': pytz.timezone('America/Phoenix'),
        'PDT': pytz.timezone('America/Los_Angeles'),
        'PST': pytz.timezone('America/Los_Angeles'),
        # Add more timezone mappings as needed
    }

    # Use the source timezone abbreviation to look up the source timezone
    source_tz = timezone_mapping.get(source_timezone)
    if source_tz is None:
        return None

    # Use a default date (you can adjust as needed)
    date_str = '1970-01-01 ' + time_str
    try:
        source_time = datetime.strptime(date_str, '%Y-%m-%d %I:%M%p')
        source_time = source_tz.localize(source_time)

        # Convert to Eastern Standard Time (EST)
        est_time = source_time.astimezone(pytz.timezone('US/Eastern'))
        return est_time.strftime('%I:%M%p %Z')
    except ValueError:
        return None

dept_times = data['Departure_Time'].to_list()
arr_times = data['Arrival_Time'].to_list()
dept_est_times = []
arr_est_times = []

for entry in dept_times:
    output_time = entry.split(' ')
    est_time = convert_to_est(output_time[0], output_time[1])
    dept_est_times.append(est_time)
for entry in arr_times:
    output_time = entry.split(' ')
    est_time = convert_to_est(output_time[0], output_time[1])
    arr_est_times.append(est_time)



data['Departure_Time'] = dept_est_times
data['Arrival_Time'] = arr_est_times

data

{'CDT', 'AST', 'EST', 'PDT', 'EDT', 'MDT', 'HST', 'MST'}


,Flight_Number,Aircraft_Type,Origin_Airport,Destination_Airport,Departure_Time,Arrival_Time,Origin_Airport_Code,Destination_Airport_Code
1,SWA1003,B737,Baltimore/Washington Intl (KBWI),San Antonio Intl (KSAT),07:22PM EST,10:59PM EST,BWI,SAT
2,SWA1005,B38M,Denver Intl (KDEN),Sacramento Intl (KSMF),06:14PM EST,08:36PM EST,DEN,SMF
3,SWA1018,B737,Cancun Int'l (CUN / MMUN),Denver Intl (KDEN),04:00PM EST,08:39PM EST,ancun Int'l (CUN / MMUN,DEN
4,SWA1036,B738,Los Angeles Intl (KLAX),Kahului (OGG / PHOG),04:16PM EST,08:31PM EST,LAX,ahului (OGG / PHOG
5,SWA1039,B737,Daugherty Field (KLGB),Dallas Love Fld (KDAL),06:17PM EST,08:45PM EST,LGB,DAL
...,...,...,...,...,...,...,...,...
436,SWA946,B738,Savannah/Hilton Head Intl (KSAV),Baltimore/Washington Intl (KBWI),07:02PM EST,08:18PM EST,SAV,BWI
437,SWA949,B737,Reagan National (KDCA),Kansas City Intl (KMCI),05:46PM EST,08:06PM EST,DCA,MCI
438,SWA954,B737,Indianapolis Intl (KIND),Phoenix Sky Harbor Intl (KPHX),07:17PM EST,09:38PM EST,IND,PHX
439,SWA956,B738,Austin-Bergstrom Intl (KAUS),Fort Lauderdale Intl (KFLL),07:25PM EST,09:32PM EST,AUS,FLL


In [59]:
# Remove "EST" from the entries
data['Dept Cleaned Time'] = data['Departure_Time'].str.replace(' EST', '')
data['Arr Cleaned Time'] = data['Arrival_Time'].str.replace(' EST', '')
# Convert the cleaned string to datetime objects
data['Departure_Time'] = pd.to_datetime(data['Dept Cleaned Time'], format='%I:%M%p')
data['Arrival_Time'] = pd.to_datetime(data['Arr Cleaned Time'], format='%I:%M%p')
# Drop the intermediate column 'Cleaned Time' if not needed
data.drop(columns=['Dept Cleaned Time','Arr Cleaned Time'], inplace=True)

In [80]:
# extract all errored Airport IATA Codes

data['Origin_Airport_Code'] = data['Origin_Airport_Code'].str.replace('aniel  Inouye Intl (PHNL', 'HNL')
data['Destination_Airport_Code'] = data['Destination_Airport_Code'].str.replace('aniel  Inouye Intl (PHNL', 'HNL')
data['Origin_Airport_Code'] = data['Origin_Airport_Code'].str.replace('ahului (OGG / PHOG', 'OGG')
data['Destination_Airport_Code'] = data['Destination_Airport_Code'].str.replace('ahului (OGG / PHOG', 'OGG')
data['Origin_Airport_Code'] = data['Origin_Airport_Code'].str.replace('OA / PHO', 'KOA')
data['Destination_Airport_Code'] = data['Destination_Airport_Code'].str.replace('OA / PHO', 'KOA')

filtered_entries = data['Origin_Airport_Code'][data['Origin_Airport_Code'].apply(lambda x: len(x) > 3)].tolist()
print(filtered_entries)

# apply additional data substitutions 
data['Origin_Airport_Code'] = data['Origin_Airport_Code'].str.replace('ihue (LIH / PHLI', 'LIH')
data['Destination_Airport_Code'] = data['Destination_Airport_Code'].str.replace('ihue (LIH / PHLI', 'LIH')
data['Origin_Airport_Code'] = data['Origin_Airport_Code'].str.replace('ancun Int\'l (CUN / MMUN', 'CUN')
data['Destination_Airport_Code'] = data['Destination_Airport_Code'].str.replace('ancun Int\'l (CUN / MMUN', 'CUN')
data['Origin_Airport_Code'] = data['Origin_Airport_Code'].str.replace('uis Munoz Marin Intl (SJU / TJSJ', 'SJU')
data['Destination_Airport_Code'] = data['Destination_Airport_Code'].str.replace('uis Munoz Marin Intl (SJU / TJSJ', 'SJU')

filtered_entries = data['Origin_Airport_Code'][data['Origin_Airport_Code'].apply(lambda x: len(x) > 3)].tolist()
print(filtered_entries)

[]
[]


In [81]:
data = data.sort_values(by='Departure_Time')
data

,Flight_Number,Aircraft_Type,Origin_Airport,Destination_Airport,Departure_Time,Arrival_Time,Origin_Airport_Code,Destination_Airport_Code
396,SWA5274,B38M,Phoenix Sky Harbor Intl (KPHX),Daniel K Inouye Intl (PHNL),1900-01-01 13:52:00,1900-01-01 20:07:00,PHX,HNL
234,SWA2698,B38M,Daniel K Inouye Intl (PHNL),Phoenix Sky Harbor Intl (KPHX),1900-01-01 13:53:00,1900-01-01 19:19:00,HNL,PHX
362,SWA4096,B38M,Kahului (OGG / PHOG),Metro Oakland Intl (KOAK),1900-01-01 14:21:00,1900-01-01 20:06:00,OGG,OA
101,SWA181,B38M,Harry Reid Intl (KLAS),Ellison Onizuka Kona Intl At Keahole (KOA / PHKO),1900-01-01 14:51:00,1900-01-01 19:40:00,LAS,KOA
397,SWA5325,B38M,Daniel K Inouye Intl (PHNL),Phoenix Sky Harbor Intl (KPHX),1900-01-01 14:58:00,1900-01-01 20:18:00,HNL,PHX
...,...,...,...,...,...,...,...,...
176,SWA2325,B737,Dallas Love Fld (KDAL),Baltimore/Washington Intl (KBWI),1900-01-01 19:52:00,1900-01-01 22:13:00,DAL,BWI
419,SWA733,B737,Kansas City Intl (KMCI),Harry Reid Intl (KLAS),1900-01-01 19:52:00,1900-01-01 22:27:00,MCI,LAS
167,SWA2252,B737,Nashville Intl (KBNA),Denver Intl (KDEN),1900-01-01 19:53:00,1900-01-01 22:29:00,BNA,DEN
361,SWA4089,B738,Denver Intl (KDEN),Albuquerque Intl Sunport (KABQ),1900-01-01 19:53:00,1900-01-01 20:48:00,DEN,ABQ


In [48]:
# Define a regular expression pattern to match the time zone code
pattern = r'([A-Z]{3})$'
# Use the str.extract method to extract the time zone codes
time_codes = data['Departure_Time'].str.extract(pattern)
time_codes_list = set(time_codes[0].tolist())
time_codes_list

{'EST'}

In [ ]:
# Transform data as a graph object for analysis
import networkx as nx

# G = (V,E)

### Analyze Total Passenger Demand to and from airports (i,j)
Data collected from NTSB

In [78]:
pass_freq = pd.read_csv('Data/Airline Dataset Updated - v2.csv')
pass_freq.columns

Index(['Passenger ID', 'First Name', 'Last Name', 'Gender', 'Age',
       'Nationality', 'Airport Name', 'Airport Country Code', 'Country Name',
       'Airport Continent', 'Continents', 'Departure Date', 'Arrival Airport',
       'Pilot Name', 'Flight Status'],
      dtype='object')